In [1]:
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from datasets.eegdataset import GeneralEEGImageDataset, MySubset
import numpy as np
import matplotlib.pyplot as plt
import torch
data_path = '/data0/tianjunchao/dataset/CVPR2021-02785/data/img_pkl/32x32'
# dataset = GeneralEEGImageDataset(path=data_path, n_channels=1, grid_size=8)
dataset = GeneralEEGImageDataset(path=data_path, n_channels=1, grid_size=8)

dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=4)

In [ ]:
len(dataset)

In [ ]:
from sklearn.model_selection import KFold, train_test_split
import torch
import torchvision.transforms as transforms
from torch.utils.data import random_split

# 定义训练集和测试集的 transforms
train_transforms = transforms.Compose([
    # rotate 30 degrees
    transforms.Resize((256, 256)),
    transforms.RandomRotation(30),
    transforms.CenterCrop(224),
    # transforms.Normalize([0.512, 0.512, 0.512], [0.228, 0.228, 0.228]),
    # transforms.RandomResizedCrop(224),
])

test_transforms = transforms.Compose([
    # rotate 30 degress
    # transforms.Resize((256, 256)),
    # # random crop 224x224
    transforms.RandomCrop(224),
    # transforms.Normalize([0.512, 0.512, 0.512], [0.228, 0.228, 0.228]),
])

# random 100 numbers from 100 to 10000
train_ids = np.random.choice(10000, 100, replace=False)
valid_ids = np.random.choice(10000, 100, replace=False)


train_dataset = MySubset(dataset, train_ids,train_transforms,target='train')
valid_dataset = MySubset(dataset, valid_ids, test_transforms)


train_loader = DataLoader(
    train_dataset, batch_size=8, num_workers=3, prefetch_factor=2)
valid_loader = DataLoader(
    valid_dataset, batch_size=8, num_workers=1, prefetch_factor=1)
  


In [ ]:
print(len(train_loader), len(valid_loader))
print(13*8)

In [ ]:
# 定义一个函数，用于展示图片
imgs = []
epochs = 2
count = 0

for data1,data2 in zip(train_loader, valid_loader):
    imgs.append((data1[0].permute(0,2,3,1),data2[0].permute(0,2,3,1)))
    count += 1
    if count == epochs:
        break

In [ ]:
indexes = 1
# draw 4 images in imgs
fig, axes = plt.subplots(1, 4, figsize=(10, 10))
for i, ax in enumerate(axes):
    ax.imshow(imgs[indexes][0][i])
    ax.set_title('train')
    ax.axis('off')
fig, axes = plt.subplots(1, 4, figsize=(10, 10))
for i, ax in enumerate(axes):
    ax.imshow(imgs[indexes][1][i])
    ax.set_title('valid')
    ax.axis('off')
plt.show()

In [2]:
# 计算均值和标准差
mean = 0.0
std = 0.0
n_batches = 0
data_stack = torch.zeros((1, 224, 224)).to('cuda')
for images, _ in dataloader:
    # print(images.shape)
    # images.shape is batchsizex3x224x224
    # only keep batchsizex1x224x224
    images = images[:, 0, :, :].squeeze().to('cuda')
    data_stack = torch.cat((data_stack, images), dim=0)
    if n_batches%10 == 0:
        print(data_stack.shape[0],torch.mean(data_stack[1:]), torch.std(data_stack[1:]))
    n_batches += 1
    # data_list.append(images)
    # batch_mean = torch.mean(images)
    # batch_std = torch.std(images)
    # mean+=batch_mean
    # std+=batch_std
    # n_batches+=1
    # if n_batches%10 == 0:
    #     # %.3f保留三位小数
    #     print('batch: %d, mean: %.3f, std: %.3f' % (n_batches, mean/n_batches, std/n_batches))
        


# mean /= len(dataset)
# std /= len(dataset)

# print("mean:", mean)
# print("std:", std)

129 tensor(0.5064, device='cuda:0') tensor(0.2301, device='cuda:0')
1409 tensor(0.5069, device='cuda:0') tensor(0.2306, device='cuda:0')
2689 tensor(0.5106, device='cuda:0') tensor(0.2306, device='cuda:0')
3969 tensor(0.5106, device='cuda:0') tensor(0.2298, device='cuda:0')
5249 tensor(0.5108, device='cuda:0') tensor(0.2295, device='cuda:0')
6529 tensor(0.5116, device='cuda:0') tensor(0.2288, device='cuda:0')
7809 tensor(0.5121, device='cuda:0') tensor(0.2288, device='cuda:0')
9089 tensor(0.5128, device='cuda:0') tensor(0.2284, device='cuda:0')
10369 tensor(0.5128, device='cuda:0') tensor(0.2282, device='cuda:0')
11649 tensor(0.5124, device='cuda:0') tensor(0.2280, device='cuda:0')
12929 tensor(0.5125, device='cuda:0') tensor(0.2278, device='cuda:0')
14209 tensor(0.5128, device='cuda:0') tensor(0.2277, device='cuda:0')
15489 tensor(0.5128, device='cuda:0') tensor(0.2276, device='cuda:0')
16769 tensor(0.5128, device='cuda:0') tensor(0.2276, device='cuda:0')
18049 tensor(0.5128, device='

: 

In [ ]:
from utils.data import filter_sample, filter_two_samples


images_list = []
label_list = []

# 创建一个字典来保存每个标签出现的次数
label_counts = {}

sum_label = 0
for i, (raw_image, raw_labels) in enumerate(dataloader):

    image, labels = filter_sample(raw_image, raw_labels)
    sum_label += len(labels)
    label = labels.numpy()
    for j in range(len(label)):
        
        # if label_counts.get(label[j], 0) >= TARGET_COUNT:
        #     continue
        # 如果标签还没有出现过两次，将元素添加到结果列表中
        # result.append((image1[j],image2[j], label[j]))
        label_counts[label[j]] = label_counts.get(label[j], 0) + 1
    # 如果每个类别的样本都出现了目标次数，跳出循环
    print('epoch',i,'label_counts',len(label_counts),'sum_label',sum_label)
print(label_counts)